##### Copyright 2021 The TensorFlow Federated Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 调整推荐的学习聚合

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/federated/tutorials/tuning_recommended_aggregators"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/federated/tutorials/tuning_recommended_aggregators.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

`tff.learning` 模块包含多种使用建议的默认配置聚合模型更新的方式：

- `tff.learning.robust_aggregator`
- `tff.learning.dp_aggregator`
- `tff.learning.compression_aggregator`
- `tff.learning.secure_aggregator`

在本教程中，我们将解释底层动机、如何实现它们，并就如何自定义它们的配置提供建议。 

---

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [1]:
import math
import tensorflow_federated as tff
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

聚合方法由对象表示，这些对象可以作为其 `model_aggregator` 关键字参数传递给 `tff.learning.algorithms.build_weighted_fed_avg`（以及 `build_unweighted_fed_avg`）。因此，这里讨论的聚合器可以直接用于修改[上一个](federated_learning_for_image_classification.ipynb)联合学习[教程](federated_learning_for_text_generation.ipynb)。

来自 [FedAvg](http://proceedings.mlr.press/v54/mcmahan17a/mcmahan17a.pdf) 算法的基线加权平均值可以使用 `tff.aggregators.MeanFactory` 表示如下：

```
mean = tff.aggregators.MeanFactory()
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    ...,
    model_aggregator=mean)
```

可用于扩展本教程中介绍的加权平均值的技术包括：

- 归零
- 裁剪
- 差分隐私
- 压缩
- 安全聚合

此扩展程序使用组合来完成，在组合中，`MeanFactory` 会包装一个内部工厂，它将聚合的某一部分委托给该工厂，或者由另一个聚合工厂包装自己。有关此设计的详细信息，请参阅[实现自定义聚合器](custom_aggregators.ipynb)教程。

首先，我们将解释如何单独启用和配置这些技术，然后展示如何将它们组合在一起。

## 技术

在深入研究各个技术之前，我们首先介绍分位数匹配算法，该算法对于配置下面的技术很有用。

### 分位数匹配

下面的几种聚合技术需要使用控制聚合的某些方面的范数界限。此类界限可以作为常量提供，但通常在训练过程中调整界限更好。推荐的方式是使用 [Andrew 等人 (2019)](https://arxiv.org/abs/1905.03871) 提出的分位数匹配算法，该算法最初提出是因为它与差分隐私的兼容性，但使用更加广泛。要估算给定分位数的值，可以使用 `tff.aggregators.PrivateQuantileEstimationProcess`。例如，要适应分布的中位数，可以使用：

In [ ]:
median_estimate = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0, target_quantile=0.5, learning_rate=0.2)

正如我们将看到的，使用分位数估算算法的不同技术将需要不同的算法参数值。通常，增加 `learning_rate` 参数意味着更快地适应到正确的分位数，但具有更高的方差。 `no_noise` 类方法能够构建一个不会为差分隐私添加噪声的分位数匹配过程。

### 归零

归零是指用零替换异常大的值。在这里，“异常大”可能意味着大于预定义的阈值，或者相对于前几轮计算中的值较大。归零可以提高系统对故障客户端上的数据损坏的稳健性。

为了计算 L 无穷范数大于 `ZEROING_CONSTANT` 归零的平均值，我们用执行归零的 `tff.aggregators.zeroing_factory` 包装 `tff.aggregators.MeanFactory`。

```
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=tff.aggregators.MeanFactory())
```

在这里，我们用 `zeroing_factory` 包装 `MeanFactory`，因为我们希望在通过求平均值将值传递给内部的 `MeanFactory` 之前，将 `zeroing_factory` 的（预聚合）效果应用于客户端的值。

然而，对于大多数应用，我们建议使用分位数估算器进行自适应归零。为此，我们使用如下分位数匹配算法：

In [ ]:
zeroing_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=10.0,
    target_quantile=0.98,
    learning_rate=math.log(10),
    multiplier=2.0,
    increment=1.0)
zeroing_mean = tff.aggregators.zeroing_factory(
    zeroing_norm=zeroing_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# zeroing_mean = tff.learning.robust_aggregator(clipping=False)

选择这些参数是为了使进程能够非常快速地适应（相对较大的 `learning_rate`），以适应比目前所见的最大值稍大的值。对于分位数估算 `Q`，用于归零的阈值将是 `Q * multiplier + increment`。

### 剪裁到限定的 L2 范数

剪裁客户端更新（投影到 L2 球上）可以提高对异常值的稳健性。`tff.aggregators.clipping_factory` 的结构与上面讨论的 `tff.aggregators.zeroing_factory` 完全相同，可以使用常量或 `tff.templates.EstimationProcess` 作为其 `clipping_norm` 参数。推荐的最佳做法是使用适度快速地适应中等高度范数的剪裁，如下所示：

In [ ]:
clipping_norm = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=1.0,
    target_quantile=0.8,
    learning_rate=0.2)
clipping_mean = tff.aggregators.clipping_factory(
    clipping_norm=clipping_norm,
    inner_agg_factory=tff.aggregators.MeanFactory())

# Equivalent to:
# clipping_mean = tff.learning.robust_aggregator(zeroing=False)

根据我们处理许多问题的经验，只要适当调整学习率，`target_quantile` 的精确值似乎并不太重要。然而，与不使用裁剪相比，将其设置得很低可能需要提高服务器学习率才能获得最佳性能，因此，我们建议默认使用 0.8。

### 差分隐私

使用自适应裁剪和高斯噪声，TFF 还支持差分隐私聚合。执行差分隐私平均的聚合器可以用如下方式进行构造：

In [ ]:
dp_mean = tff.aggregators.DifferentiallyPrivateFactory.gaussian_adaptive(
    noise_multiplier=0.1, clients_per_round=100)

# Equivalent to:
# dp_mean = tff.learning.dp_aggregator(
#   noise_multiplier=0.1, clients_per_round=100, zeroing=False)

有关如何设置 `noise_multiplier` 参数的指导，请参阅 [TFF DP 教程](https://tensorflow.google.cn/federated/tutorials/federated_learning_with_differential_privacy)。

### 有损压缩

与无损压缩（如 GZIP）相比，有损压缩通常会产生更高的压缩比，并且之后仍可与无损压缩结合使用。由于在客户端到服务器通信上花费的时间更少，因此训练周期完成得更快。由于学习算法本身的随机性，在一定阈值下，有损压缩带来的误差不会对整体性能产生负面影响。

默认的建议是使用简单的统一量化（例如，请参阅 [Suresh 等人](http://proceedings.mlr.press/v70/suresh17a/suresh17a.pdf)），该量化由两个值来参数化：张量大小压缩 `threshold` 和 `quantization_bits` 的数量。对于每个张量 `t`，如果 `t` 的元素数小于或等于 `threshold`，则它不会被压缩。如果较大，则使用随机舍入将 `t` 的元素量化为 `quantizaton_bits` 比特。也就是说，我们会应用运算

`t = round((t - min(t)) / (max(t) - min(t)) * (2**quantizaton_bits - 1)),`

从而得到范围为 `[0, 2**quantizaton_bits-1]` 的整数值。将量化后的值直接打包成整数类型进行传输，然后进行逆变换。

我们建议将 `quantizaton_bits` 设置为等于 8，并将 `threshold` 设置为等于 20000：


In [ ]:
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))

# Equivalent to:
# compressed_mean = tff.learning.compression_aggregator(zeroing=False, clipping=False)

#### 调整建议

`quantization_bits` 和 `threshold` 两个参数都可以调整，每轮训练的客户端数量也会影响压缩的效果。

**阈值**。选择默认值 20000 是因为我们观察到元素数量较少的变量，如通用层类型中的偏差，对引入的噪声更敏感。此外，在实践中，对元素数量较少的变量进行压缩几乎没有什么好处，因为它们的未压缩大小从一开始就相对较小。

在某些应用程序中，更改阈值的选择可能比较有意义。例如，分类模型的输出层的偏差可能对噪声更敏感。如果您正在训练词汇量为 20004 的语言模型，您可能希望将 `threshold` 设置为20004。

**量化比特。**对于大多数用户来说，`quantization_bits` 的默认值 8 应该就足够了。如果 8 运行良好，而您想要挤出更多性能，您可以尝试将其降低到 7 或 6。如果资源允许进行小型网格搜索，我们建议您确定训练变得不稳定或最终模型质量开始下降的值，然后将该值增加 2。例如，如果将 `quantization_bits` 设置为 5 时有效，但将其设置为 4 时会降低模型的性能，我们建议以“安全起见”，将默认值设置为 6。

**每轮客户端数。**请注意，大幅增加每轮客户端数可以用较小的值使 `quantization_bits`很好地工作，因为量化带来的随机化不准确可以通过对更多客户端更新进行平均来抵消。

### 安全聚合

安全聚合 (SecAgg) 指的是一种加密协议，其中客户端更新以服务器只能解密其总和的方式进行加密。如果报告回来的客户端数量不足，服务器将什么也学习不到，在任何情况下，服务器都无法检查单个更新。这是使用 `tff.federated_secure_sum_bitwidth` 算子实现的。

模型更新是浮点值，但 SecAgg 会对整数进行运算。因此，在离散化为整数类型之前，我们需要将任何较大的值剪裁到某个界限。剪裁边界可以是常量，也可以通过自适应确定（推荐的默认值）。然后对整数进行安全求和，并将和映射回浮点域。

要使用 SecAgg 将 `MY_SECAGG_BOUND` 作为剪裁边界来计算加权值相加的平均值，请将 `SecureSumFactory` 传递到 `MeanFactory`，如下所示：

```
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(MY_SECAGG_BOUND))
```

要在自适应确定边界时执行相同的操作，请执行以下代码：

In [ ]:
secagg_bound = tff.aggregators.PrivateQuantileEstimationProcess.no_noise(
    initial_estimate=50.0,
    target_quantile=0.95,
    learning_rate=1.0,
    multiplier=2.0)
secure_mean = tff.aggregators.MeanFactory(
    tff.aggregators.SecureSumFactory(secagg_bound))

# Equivalent to:
# secure_mean = tff.learning.secure_aggregator(zeroing=Fasle, clipping=False)

#### 调整建议

自适应参数的选择使得边界很紧密（我们在离散化中不会损失太多精度），但剪裁很少发生。

如果调整参数，请记住，SecAgg 协议是在加权平均值之后对加权模型更新求和。权重通常是本地处理的数据点的数量，因此在不同的任务之间，正确的界限可能取决于此数量。

我们不建议在创建自适应 `secagg_bound` 时使用 `increment` 关键字参数，因为这可能会在实际估计很小的情况下，导致较大的相对精度损失。

上面的代码段将只使用 SecAgg 加权值。如果还应将 SecAgg 用于权重总和，我们建议将界限设置为常量。在常见的训练设置中，会预先知道可能的最大权重：

```
secure_mean = tff.aggregators.MeanFactory(
    value_sum_factory=tff.aggregators.SecureSumFactory(secagg_bound),
    weight_sum_factory=tff.aggregators.SecureSumFactory(
        upper_bound_threshold=MAX_WEIGHT, lower_bound_threshold=0.0))
```

## 组合技术

用于扩展上面介绍的平均值的各个技术可以组合在一起。

我们建议在客户端应用这些技术的顺序为：

1. 归零
2. 裁剪
3. 其他技术

`tff.aggregators` 模块中的聚合器是通过在“外部聚合器”内部包装“内部聚合器”（聚合前效应最后发生，聚合后效应最先发生）组成的。例如，要（按上述顺序）执行归零、裁剪和压缩，应编写：

```
# Compression is innermost because its pre-aggregation effects are last.
compressed_mean = tff.aggregators.MeanFactory(
    tff.aggregators.EncodedSumFactory.quantize_above_threshold(
        quantization_bits=8, threshold=20000))
# Compressed mean is inner aggregator to clipping...
clipped_compressed_mean = tff.aggregators.clipping_factory(
    clipping_norm=MY_CLIPPING_CONSTANT,
    inner_agg_factory=compressed_mean)
# ...which is inner aggregator to zeroing, since zeroing happens first.
final_aggregator = tff.aggregators.zeroing_factory(
    zeroing_norm=MY_ZEROING_CONSTANT,
    inner_agg_factory=clipped_compressed_mean)
```

请注意，此结构与学习算法的[默认聚合器](https://github.com/tensorflow/federated/blob/11e4f632b38745c9b38cc39fa1fe67771c206e77/tensorflow_federated/python/learning/model_update_aggregator.py)匹配。

还可以使用其他组合。当我们确信可以提供在多个不同应用程序中工作的默认配置时，我们会扩展本文档。有关实现新想法的信息，请参阅[实现自定义聚合器](custom_aggregators.ipynb)教程。